# Loading models

In [61]:
import keras
from keras.models import Model

In [62]:
rgb_model = keras.models.load_model('checkpoints/weights_i3drgb.hdf5')
flow_model = keras.models.load_model('checkpoints/weights_i3dflow.hdf5')

In [63]:
rgb_model.pop()
flow_model.pop()

In [64]:
rgb_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i3d_inception (Functional)   (None, None, 1, 1, 1024)  12294544  
_________________________________________________________________
dropout (Dropout)            (None, None, 1, 1, 1024)  0         
_________________________________________________________________
Conv3d_6a_1x1 (Conv3D)       (None, None, 1, 1, 2)     2048      
_________________________________________________________________
reshape (Reshape)            (None, None, 2)           0         
_________________________________________________________________
lambda (Lambda)              (None, 2)                 0         
Total params: 12,296,592
Trainable params: 2,785,520
Non-trainable params: 9,511,072
_________________________________________________________________


In [65]:
flow_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i3d_inception (Functional)   (None, None, 1, 1, 1024)  12272592  
_________________________________________________________________
dropout (Dropout)            (None, None, 1, 1, 1024)  0         
_________________________________________________________________
Conv3d_6a_1x1 (Conv3D)       (None, None, 1, 1, 2)     2048      
_________________________________________________________________
reshape (Reshape)            (None, None, 2)           0         
_________________________________________________________________
lambda (Lambda)              (None, 2)                 0         
Total params: 12,274,640
Trainable params: 2,785,520
Non-trainable params: 9,489,120
_________________________________________________________________


# Data generator

In [66]:
import numpy as np
from keras.utils import Sequence
from keras.utils import np_utils
import os
import random
import math

In [67]:
class DataGenerator(Sequence):
    """Data Generator inherited from keras.utils.Sequence
    Args: 
        directory: the path of data set, and each sub-folder will be assigned to one class
        batch_size: the number of data points in each batch
        shuffle: whether to shuffle the data per epoch
    Note:
        If you want to load file with other data format, please fix the method of "load_data" as you want
    """
    def __init__(self, directory, batch_size=1, shuffle=True, data_augmentation=True, 
                 target_frames = 79, crop_dim = (224, 224), seed = None, flip = True):
        # Initialize the params
        self.batch_size = batch_size
        self.directory = directory
        self.shuffle = shuffle
        self.data_aug = data_augmentation
        self.target_frames = target_frames
        self.seed = seed
        self.crop_dim = crop_dim
        self.flip = True
        # Load all the save_path of files, and create a dictionary that save the pair of "data:label"
        self.X_path, self.Y_dict = self.search_data() 
        # Print basic statistics information
        self.print_stats()
        return None
    
    def search_data(self):
        X_path = []
        Y_dict = {}
        # list all kinds of sub-folders
        self.dirs = sorted(os.listdir(self.directory))
        one_hots = np_utils.to_categorical(range(len(self.dirs)))
        for i,folder in enumerate(self.dirs):
            folder_path = os.path.join(self.directory,folder)
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path,file)
                # append the each file path, and keep its label  
                X_path.append(file_path)
                Y_dict[file_path] = one_hots[i]
        return X_path, Y_dict
    
    def print_stats(self):
        # calculate basic information
        self.n_files = len(self.X_path)
        self.n_classes = len(self.dirs)
        self.indexes = np.arange(len(self.X_path))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
        # Output states
        print("Found {} files belonging to {} classes.".format(self.n_files,self.n_classes))
        for i,label in enumerate(self.dirs):
            print('%10s : '%(label),i)
        return None
    
    def __len__(self):
        # calculate the iterations of each epoch
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)
    
    def __getitem__(self, index):
        """Get the data of each batch
        """
        # get the indexs of each batch
        batch_indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # using batch_indexs to get path of current batch
        batch_path = [self.X_path[k] for k in batch_indexs]
        # get batch data
        batch_x, batch_y = self.data_generation(batch_path)
        return batch_x, batch_y
    
    def on_epoch_end(self):
        # shuffle the data at each end of epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def data_generation(self, batch_path):
        # load data into memory, you can change the np.load to any method you want        
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]
        # transfer the data format and take one-hot coding for labels
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y
    
    def dynamic_crop(self, video):
        video_dim = video.shape
        video_width = video_dim[2]
        video_height = video_dim[3]
        
        if self.data_aug:
            x_max = video_width - self.crop_dim[0]
            y_max = video_height - self.crop_dim[1]

            x = random.randint(0, x_max)
            y = random.randint(0, y_max)
            
        else:
            x_center = math.ceil(video_width/2)
            y_center = math.ceil(video_height/2)
            
            x = x_center - math.ceil(self.crop_dim[0]/2)
            y = y_center - math.ceil(self.crop_dim[1]/2)
                        
        return video[:,:,x:x+self.crop_dim[0],y:y+self.crop_dim[1],:]
        
    
    def frame_sampling(self, video):
        # get total frames of input video
        len_frames = video.shape[1]
        
        # If the video is shorter than needed
        if len_frames < self.target_frames:
            # Times the video need to be looped to get 64 frames
            times = self.target_frames//len_frames
            remainder = self.target_frames%len_frames
            # Creating new array to store cat video
            new_video = video
            
            # Repeat the video as many times as needed
            for n in range(1,times):
                new_video = np.concatenate((new_video, video), axis = 1)
            # Add part of the video if needed
            if remainder > 0:
                new_video = np.concatenate((new_video, video[:,:remainder,:,:]), axis = 1)
            
            return new_video
               
        # If the video is longer than needed
        elif len_frames > self.target_frames:
            # Set random start
            start_frame = random.randint(0,len_frames - self.target_frames)
            end_frame = start_frame + self.target_frames
            
            new_video = video[:,start_frame:end_frame,:,:]
            
            return new_video
        
        # If the video is fine
        elif len_frames == self.target_frames:
            return video
    
    def random_flip(self, video): 
        # Flip on width (left-rigth)
        if random.randint(0,1) == 1:
            video = np.flip(video, axis = 3)
                
        return video
    
    def load_data(self, path):
        data = np.load(path)['arr_0']
    
        # Sampling frames
        if self.target_frames is not None:
            data = self.frame_sampling(video = data)
        
        # Center if data_aug is false and random if data_aug is true
        data = self.dynamic_crop(data)
        
        # If it needs flip
        if self.flip:
            data = self.random_flip(data)

        return data[0]

In [68]:
batch_size = 8
rgb_path_train = '../datai3d/rgb/train/'
rgb_path_val = '../datai3d/rgb/validation/'
rgb_path_test = '../datai3d/rgb/test/'

flow_path_train = '../datai3d/flow/train/'
flow_path_val = '../datai3d/flow/validation/'
flow_path_test = '../datai3d/flow/test/'

In [69]:
rgb_train_generator = DataGenerator(directory=rgb_path_train, 
                                shuffle = False,
                                batch_size=batch_size, 
                                data_augmentation=True)

rgb_validation_generator = DataGenerator(directory=rgb_path_val,
                                     shuffle = False,
                                     batch_size=batch_size,
                                     data_augmentation=False,
                                     target_frames = None,
                                     flip = False)

rgb_test_generator = DataGenerator(directory=rgb_path_test,
                               shuffle = False,
                                batch_size=batch_size, 
                                data_augmentation=False,
                                target_frames = None,
                                flip = False)

flow_train_generator = DataGenerator(directory=flow_path_train, 
                                shuffle = False,
                                batch_size=batch_size, 
                                data_augmentation=True)

flow_validation_generator = DataGenerator(directory=flow_path_val,
                                     shuffle = False,
                                     batch_size=batch_size,
                                     data_augmentation=False,
                                     target_frames = None,
                                     flip = False)

flow_test_generator = DataGenerator(directory=flow_path_test,
                               shuffle = False,
                                batch_size=batch_size, 
                                data_augmentation=False,
                                target_frames = None,
                                flip = False)

Found 1207 files belonging to 2 classes.
     Fight :  0
  NonFight :  1
Found 393 files belonging to 2 classes.
     Fight :  0
  NonFight :  1
Found 400 files belonging to 2 classes.
     Fight :  0
  NonFight :  1
Found 1207 files belonging to 2 classes.
     Fight :  0
  NonFight :  1
Found 393 files belonging to 2 classes.
     Fight :  0
  NonFight :  1
Found 400 files belonging to 2 classes.
     Fight :  0
  NonFight :  1


In [70]:
rgb_predictions = rgb_model.predict(x = rgb_test_generator,
                                    steps = rgb_test_generator.n_files//batch_size,
                                   verbose = 1)

50/50 [==============================] - 384s 8s/step


In [71]:
flow_predictions = flow_model.predict(x = flow_test_generator,
                                    steps = flow_test_generator.n_files//batch_size,
                                   verbose = 1)

50/50 [==============================] - 328s 7s/step


In [72]:
predictions = (rgb_predictions + flow_predictions).argmax(-1)

In [73]:
predictions

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,

In [86]:
list(rgb_test_generator.Y_dict.keys())

['../datai3d/rgb/test/Fight\\0Ow4cotKOuw_0_rgb.npz',
 '../datai3d/rgb/test/Fight\\0Ow4cotKOuw_1_rgb.npz',
 '../datai3d/rgb/test/Fight\\0Ow4cotKOuw_2_rgb.npz',
 '../datai3d/rgb/test/Fight\\0Ow4cotKOuw_3_rgb.npz',
 '../datai3d/rgb/test/Fight\\0Ow4cotKOuw_4_rgb.npz',
 '../datai3d/rgb/test/Fight\\1Kbw1bUw_0_rgb.npz',
 '../datai3d/rgb/test/Fight\\1Kbw1bUw_1_rgb.npz',
 '../datai3d/rgb/test/Fight\\1Kbw1bUw_2_rgb.npz',
 '../datai3d/rgb/test/Fight\\1Kbw1bUw_3_rgb.npz',
 '../datai3d/rgb/test/Fight\\1MVS2QPWbHc_0_rgb.npz',
 '../datai3d/rgb/test/Fight\\1MVS2QPWbHc_1_rgb.npz',
 '../datai3d/rgb/test/Fight\\1MVS2QPWbHc_2_rgb.npz',
 '../datai3d/rgb/test/Fight\\1MVS2QPWbHc_3_rgb.npz',
 '../datai3d/rgb/test/Fight\\1MVS2QPWbHc_4_rgb.npz',
 '../datai3d/rgb/test/Fight\\1MVS2QPWbHc_5_rgb.npz',
 '../datai3d/rgb/test/Fight\\39BFeYnbu-I_0_rgb.npz',
 '../datai3d/rgb/test/Fight\\39BFeYnbu-I_1_rgb.npz',
 '../datai3d/rgb/test/Fight\\39BFeYnbu-I_2_rgb.npz',
 '../datai3d/rgb/test/Fight\\39BFeYnbu-I_3_rgb.npz',
 '../

In [74]:
ground_truth = list(rgb_test_generator.Y_dict.values())

In [76]:
ground_truth = np.array([labels.argmax(-1) for labels in ground_truth])

In [77]:
ground_truth

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [78]:
from sklearn import metrics

In [80]:
print(metrics.classification_report(ground_truth, predictions))

              precision    recall  f1-score   support

           0       0.76      0.83      0.80       200
           1       0.82      0.73      0.77       200

    accuracy                           0.79       400
   macro avg       0.79      0.78      0.78       400
weighted avg       0.79      0.79      0.78       400



In [81]:
metrics.accuracy_score(ground_truth, predictions)

0.785